In [ ]:
import cv2
import matplotlib.pyplot as plt
import imgaug.augmenters as aug
import random
import shutil
import os
import time
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from google.colab import drive

In [ ]:
!mkdir -p sampah/kosong sampah/penuh sampah/setengah_penuh

In [ ]:
data = './sampah/'

In [ ]:
save = './sampah/'

In [ ]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    rescale = 1. / 255,
    zoom_range = 0.2
    )

test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(
    'sampah',
    target_size = (150, 150),
    batch_size = 32,
    class_mode='categorical'
)
validation_generator = test_datagen.flow_from_directory(
        'sampah',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')


Found 1 images belonging to 3 classes.
Found 1 images belonging to 3 classes.


In [ ]:
for i in range(0, 100):
  x, y = next(train_generator)

/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


In [ ]:
train_dir = r''
val_dir = r''
test_dir = r''

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical')

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if (logs.get('accuracy') > 0.90):
      print('\nAkurasi sudah bagus')
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer = 'adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
model = create_model()

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=2000,
                    epochs=15,
                    validation_data=validation_generator,
                    validation_steps=800,
                    callbacks = [callbacks]
)

In [ ]:
epoch_avg_acc = sum(history.history['accuracy']) / len(history.history['accuracy'])
print("Epoch average accuracy: {:.2f}%".format(epoch_avg_acc * 100))

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(test_generator)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
target_names = ['Class 1', 'Class 2', 'Class 3']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("test loss:",test_loss)
print("test acc:",test_accuracy)

In [ ]:
cm = confusion_matrix(test_generator.classes, y_pred)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_xticklabels(target_names)
ax.set_yticklabels(target_names)
plt.show()